# exp034

In [1]:
import os
import sys
import gc
import itertools
import pickle
import pathlib
import datetime
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
load_dotenv()
sys.path.append(os.getenv('UTILS_PATH'))

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

import line_notify

In [2]:
import builtins
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

In [3]:
SEED = 42
N_ITER = 1
RUN_INF = False # 推論処理を行うか
BATCH_SIZE = int(5e5)

In [4]:
Ns = {}
Ns['cf_a'] = 12
Ns['ctf_a'] = 12
Ns['atfd_a'] = 12
Ns['atfp_a'] = 12
Ns['pa_a'] = 12

Ns['cf_w'] = 12
Ns['ctf_w'] = 12
Ns['atfd_w'] = 12
Ns['atfp_w'] = 12
Ns['pa_w'] = 12

Ns['cf_m'] = 12
Ns['ctf_m'] = 12
Ns['atfd_m'] = 12
Ns['atfp_m'] = 12
Ns['pa_m'] = 12

Ns['cf_y'] = 12
Ns['ctf_y'] = 12
Ns['atfd_y'] = 12
Ns['atfp_y'] = 12
Ns['pa_y'] = 12

ディレクトリ設定

In [5]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
#exp_name = os.path.dirname(__file__).split('/')[-1]
#exp_name = 'exp034'
#os.makedirs(OUTPUT_DIR + exp_name, exist_ok=True)

データ読み込み

In [6]:
articles = pd.read_csv(INPUT_DIR + 'articles.csv', dtype='object')
customers = pd.read_csv(INPUT_DIR + 'customers.csv')
transactions = pd.read_csv(INPUT_DIR + 'transactions_train.csv', dtype={'article_id':'str'}, parse_dates=['t_dat'])
sample = pd.read_csv(INPUT_DIR + 'sample_submission.csv')

# 前処理

In [7]:
ALL_CUSTOMER = customers['customer_id'].unique().tolist()
ALL_ARTICLE = articles['article_id'].unique().tolist()

customer_ids = dict(list(enumerate(ALL_CUSTOMER)))
article_ids = dict(list(enumerate(ALL_ARTICLE)))

customer_map = {u: uidx for uidx, u in customer_ids.items()}
article_map = {i: iidx for iidx, i in article_ids.items()}

articles['article_id'] = articles['article_id'].map(article_map)
customers['customer_id'] = customers['customer_id'].map(customer_map)
transactions['article_id'] = transactions['article_id'].map(article_map)
transactions['customer_id'] = transactions['customer_id'].map(customer_map)
sample['customer_id'] = sample['customer_id'].map(customer_map)

In [8]:
# 名寄せ
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].str.replace('None','NONE')

In [9]:
customers['age10'] = str((customers['age'] // 10) * 10)
customers.loc[customers['age'].isnull(), 'age10'] = np.nan

In [10]:
# label_encoding
le_cols = ['product_type_name', 'product_group_name', 'graphical_appearance_name',
            'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name', 'department_name',
            'index_name', 'index_group_name', 'section_name', 'garment_group_name']
for c in le_cols:
    le = LabelEncoder()
    articles[c] = le.fit_transform(articles[c].fillna(''))


le_cols = ['club_member_status', 'fashion_news_frequency', 'postal_code', 'age10']
for c in le_cols:
    le = LabelEncoder()
    customers[c] = le.fit_transform(customers[c].fillna(''))

In [11]:
customers['customer_type'] = customers['FN'].fillna(0).astype(int).astype(str) + \
                             customers['Active'].fillna(0).astype(int).astype(str) + \
                             customers['club_member_status'].fillna(0).astype(int).astype(str) + \
                             customers['fashion_news_frequency'].fillna(0).astype(int).astype(str) + \
                             customers['age10'].fillna(0).astype(int).astype(str)

le = LabelEncoder()
customers['customer_type'] = le.fit_transform(customers['customer_type'])

In [12]:
# transactionに紐づけ
transactions = transactions.merge(customers, on='customer_id', how='left')
transactions = transactions.merge(articles, on='article_id', how='left')

# データセット作成（レコメンド→対象データセット作成→特徴量エンジニアリング）

In [13]:
@noglobal
def get_customer_frequent(history, n=12, timedelta=None):
    """顧客ごと商品の購入数をカウントし上位の商品を抽出

    Args:
        history (dataframe): 集計対象の実績データ
        n (int): レコメンド対象とする数
        timedelta (dateutil.relativedelta): 指定された場合、実績データの終端からtimedelta分のデータを取得する

    Returns:
        dataframe: 抽出結果
    """
    if timedelta is not None:
        st_date = history['t_dat'].max() - timedelta
        history = history[history['t_dat']>=st_date].copy()
        
    customer_agg = history.groupby(['customer_id', 'article_id'])['t_dat'].count().reset_index()
    customer_agg = customer_agg.rename(columns={'t_dat':'cnt'})
    customer_agg = customer_agg.sort_values(['customer_id', 'cnt'], ascending=False)
    result = customer_agg.groupby('customer_id').head(n)
    return result[['customer_id', 'article_id']]

@noglobal
def get_popular_article(history, n=12, timedelta=None):
    """全体の購入数をカウントし上位の商品を抽出

    Args:
        history (dataframe): 集計対象の実績データ
        n (int): レコメンド対象とする数
        timedelta (dateutil.relativedelta): 指定された場合、実績データの終端からtimedelta分のデータを取得する

    Returns:
        list: 抽出結果
    """
    # 全体の購入数量
    if timedelta is not None:
        st_date = history['t_dat'].max() - timedelta
        history = history[history['t_dat']>=st_date].copy()

    total_agg = history.groupby('article_id')['t_dat'].count().reset_index()
    total_agg = total_agg.rename(columns={'t_dat':'cnt'})
    total_agg = total_agg.sort_values(['cnt'], ascending=False)
    total_agg = total_agg.head(n)
    result = list(total_agg['article_id'].values)
    return result

@noglobal
def get_customer_type_frequent(history, n=12, timedelta=None):
    if timedelta is not None:
        st_date = history['t_dat'].max() - timedelta
        history = history[history['t_dat']>=st_date].copy()

    result = history[['customer_id', 'customer_type']].drop_duplicates().copy()
    agg = history.groupby(['customer_type', 'article_id'])['t_dat'].count().reset_index()
    agg = agg.rename(columns={'t_dat':'cnt'})
    agg = agg.sort_values(['customer_type', 'cnt'], ascending=False)
    agg = agg.groupby('customer_type').head(n)
    result = result.merge(agg[['customer_type', 'article_id']], on='customer_type', how='left')
    return result[['customer_id', 'article_id']]

@noglobal
def get_article_type_frequent(history, col, n=12, timedelta=None):
    if timedelta is not None:
        st_date = history['t_dat'].max() - timedelta
        history = history[history['t_dat']>=st_date].copy()

    result = history.groupby(['customer_id', col])['t_dat'].count().reset_index()
    result = result.rename(columns={'t_dat':'cnt'})
    result = result.sort_values(['customer_id', 'cnt'], ascending=False)
    result = result.groupby(['customer_id']).head(1)[['customer_id', col]]

    agg = history.groupby([col, 'article_id'])['t_dat'].count().reset_index()
    agg = agg.rename(columns={'t_dat':'cnt'})
    agg = agg.sort_values([col, 'cnt'], ascending=False)
    agg = agg.groupby(col).head(n)
    result = result.merge(agg[[col, 'article_id']], on=col, how='left')
    return result[['customer_id', 'article_id']]

@noglobal
def get_reccomend(target_customer_id, history, Ns):
    result = pd.DataFrame()
    
    td = None
    result = result.append(get_customer_frequent(history, Ns['cf_a'], td))
    result = result.append(get_customer_type_frequent(history, Ns['ctf_a'], td))
    result = result.append(get_article_type_frequent(history, 'department_name', Ns['atfd_a'], td))
    result = result.append(get_article_type_frequent(history, 'perceived_colour_master_name', Ns['atfp_a'], td))
    popular_article = get_popular_article(history, Ns['pa_a'], td)
    # customerとpopular articleの全組み合わせでdataframe作成
    popular_article = pd.DataFrame(itertools.product(target_customer_id, popular_article), columns=['customer_id', 'article_id'])
    result = result.append(popular_article)
    result = result.drop_duplicates()

    td = relativedelta(weeks=1)
    result = result.append(get_customer_frequent(history, Ns['cf_w'], td))
    result = result.append(get_customer_type_frequent(history, Ns['ctf_w'], td))
    result = result.append(get_article_type_frequent(history, 'department_name', Ns['atfd_w'], td))
    result = result.append(get_article_type_frequent(history, 'perceived_colour_master_name', Ns['atfp_w'], td))
    popular_article = get_popular_article(history, Ns['pa_w'], td)
    # customerとpopular articleの全組み合わせでdataframe作成
    popular_article = pd.DataFrame(itertools.product(target_customer_id, popular_article), columns=['customer_id', 'article_id'])
    result = result.append(popular_article)
    result = result.drop_duplicates()

    td = relativedelta(months=1)
    result = result.append(get_customer_frequent(history, Ns['cf_m'], td))
    result = result.append(get_customer_type_frequent(history, Ns['ctf_m'], td))
    result = result.append(get_article_type_frequent(history, 'department_name', Ns['atfd_m'], td))
    result = result.append(get_article_type_frequent(history, 'perceived_colour_master_name', Ns['atfp_m'], td))
    popular_article = get_popular_article(history, Ns['pa_m'], td)
    # customerとpopular articleの全組み合わせでdataframe作成
    popular_article = pd.DataFrame(itertools.product(target_customer_id, popular_article), columns=['customer_id', 'article_id'])
    result = result.append(popular_article)
    result = result.drop_duplicates()

    td = relativedelta(years=1)
    result = result.append(get_customer_frequent(history, Ns['cf_y'], td))
    result = result.append(get_customer_type_frequent(history, Ns['ctf_y'], td))
    result = result.append(get_article_type_frequent(history, 'department_name', Ns['atfd_y'], td))
    result = result.append(get_article_type_frequent(history, 'perceived_colour_master_name', Ns['atfp_y'], td))
    popular_article = get_popular_article(history, Ns['pa_y'], td)
    # customerとpopular articleの全組み合わせでdataframe作成
    popular_article = pd.DataFrame(itertools.product(target_customer_id, popular_article), columns=['customer_id', 'article_id'])
    result = result.append(popular_article)
    result = result.drop_duplicates()

    result = result[result['customer_id'].isin(target_customer_id)].copy()

    return result

In [14]:
@noglobal
def add_labels(recom_result, history):
    """レコメンドしたデータが学習期間で購入されたかどうかのフラグを付与する

    Args:
        recom_result (_type_): レコメンド結果
        train_tran (_type_): 学習期間のトランザクションデータ

    Returns:
        _type_: 学習期間での購入フラグを付与したレコメンド結果
    """
    history = history[['customer_id', 'article_id']].drop_duplicates()
    history['buy'] = 1
    recom_result = recom_result.merge(history, on=['customer_id', 'article_id'], how='left')
    recom_result['buy'] = recom_result['buy'].fillna(0)
    return recom_result


In [15]:
@noglobal
def make_article_features(articles):
    cols = ['product_type_name', 'product_group_name', 'graphical_appearance_name',
            'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name', 'department_name',
            'index_name', 'index_group_name', 'section_name', 'garment_group_name']
    return articles[['article_id']+cols]

@noglobal
def make_article_tran_features(history):
    df = history.groupby('article_id').agg({'t_dat':['count', 'max', 'min'],
                                            'price':['max', 'min', 'mean'], 
                                            'age':['max', 'min', 'mean', 'std']}).reset_index()
    df.columns = ['article_id','article_total_cnt', 'article_total_latest_buy', 'article_total_1st_buy', 'article_price_max', 'article_price_min', 'article_price_mean', 'article_age_max', 'article_age_min', 'article_age_mean', 'article_age_std']
    df['article_total_1st_buy'] = (history['t_dat'].max() - df['article_total_1st_buy']).dt.days
    df['article_total_latest_buy'] = (history['t_dat'].max() - df['article_total_latest_buy']).dt.days
    return df


@noglobal
def make_customer_features(customers):
    return customers

@noglobal
def make_customer_tran_features(history):
    df = history.groupby('customer_id').agg({'t_dat':['count', 'max', 'min'],
                                            'price':['max', 'min', 'mean']}).reset_index()
    df.columns = ['customer_id','customer_total_cnt', 'customer_total_latest_buy', 'customer_total_1st_buy', 'customer_price_max', 'customer_price_min', 'customer_price_mean']
    df['customer_total_1st_buy'] = (history['t_dat'].max() - df['customer_total_1st_buy']).dt.days
    df['customer_total_latest_buy'] = (history['t_dat'].max() - df['customer_total_latest_buy']).dt.days
    return df

@noglobal
def make_customer_article_features(target, history):
    df = target.merge(history, on=['customer_id', 'article_id'], how='inner')
    df = df.groupby(['customer_id', 'article_id']).agg({'t_dat':['count', 'min', 'max']}).reset_index()
    df.columns = ['customer_id', 'article_id', 'count', '1st_buy_date_diff', 'latest_buy_date_diff']
    df['1st_buy_date_diff'] = (history['t_dat'].max() - df['1st_buy_date_diff']).dt.days
    df['latest_buy_date_diff'] = (history['t_dat'].max() - df['latest_buy_date_diff']).dt.days
    return df

@noglobal
def add_same_article_type_rate(target, history, col):
    add_data = history[['customer_id', col]].copy()
    add_data['total'] = add_data.groupby('customer_id').transform('count')
    add_data = add_data.groupby(['customer_id', col])['total'].agg(['max', 'count']).reset_index()
    add_data[f'{col}_customer_buy_rate'] = add_data['count'] / add_data['max']
    target = target.merge(add_data[['customer_id', col, f'{col}_customer_buy_rate']], on=['customer_id', col], how='left')
    return target

    

@noglobal
def add_features(df, history, articles, customers):
    df = df.merge(make_article_features(articles), on=['article_id'], how='left')
    df = df.merge(make_article_tran_features(history), on=['article_id'], how='left')
    df = df.merge(make_customer_features(customers), on=['customer_id'], how='left')
    df = df.merge(make_customer_tran_features(history), on=['customer_id'], how='left')
    df = df.merge(make_customer_article_features(df[['customer_id', 'article_id']], history), on=['article_id', 'customer_id'], how='left')

    cols = ['product_type_name', 'product_group_name', 'graphical_appearance_name',
            'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name', 'department_name',
            'index_name', 'index_group_name', 'section_name', 'garment_group_name']

    for c in cols:
        df = add_same_article_type_rate(df, history, c)

    return df
    

# レコメンド商品を購入するかどうかの2値分類モデル

In [16]:
def apk(y_true, y_pred, K=12):
    assert(len(y_true) == len(y_pred))
    apks = []
    for idx in range(len(y_true)):
        y_i_true = y_true[idx]
        y_i_pred = y_pred[idx]

        # 予測値の数と重複の確認
        assert(len(y_i_pred) <= K)
        assert(len(np.unique(y_i_pred)) == len(y_i_pred))

        sum_precision = 0.0
        num_hits = 0.0

        for i, p in enumerate(y_i_pred):
            if p in y_i_true:
                num_hits += 1
                precision = num_hits / (i+1)
                sum_precision += precision
        apk = sum_precision / min(len(y_i_true), K)
        apks.append(apk)
    return apks

In [17]:
@noglobal
def run_train(transactions, articles, customers, Ns):
    # 1週ずつローリングして学習データを生成し検証
    train_start = datetime.datetime(2020,9,9)
    valid_start = datetime.datetime(2020,9,16)
    valid_end = datetime.datetime(2020,9,22)

    # 学習データの作成
    history_tran = transactions[transactions['t_dat'] < train_start].copy()
    target_tran = transactions[(transactions['t_dat'] >= train_start) & (transactions['t_dat'] < valid_start)].copy()
    target_id = target_tran['customer_id'].unique().tolist()
    recom = get_reccomend(target_id, history_tran, Ns)
    ml_train = add_labels(recom, target_tran)
    ml_train = add_features(ml_train, history_tran, articles, customers)

    # 評価データの作成
    history_tran = transactions[transactions['t_dat'] < valid_start].copy()
    target_tran = transactions[(transactions['t_dat'] >= valid_start) & (transactions['t_dat'] <= valid_end)].copy()
    target_id = target_tran['customer_id'].unique().tolist()
    recom = get_reccomend(target_id, history_tran, Ns)
    ml_valid = add_labels(recom, target_tran)
    ml_valid = add_features(ml_valid, history_tran, articles, customers)

    target = 'buy'
    not_use_cols = ['customer_id', 'article_id', target]
    features = [c for c in ml_train.columns if c not in not_use_cols]

    params = {
    "objective": "binary",
    "boosting" : "gbdt",
    "learning_rate": 0.01,
    "metric": "binary_logloss",
    "seed": 42
}

    # 学習
    tr_x, tr_y = ml_train[features], ml_train[target]
    vl_x, vl_y = ml_valid[features], ml_valid[target]
    tr_data = lgb.Dataset(tr_x, label=tr_y)
    vl_data = lgb.Dataset(vl_x, label=vl_y)
    model = lgb.train(params, tr_data, valid_sets=[tr_data, vl_data],
                    num_boost_round=20000, early_stopping_rounds=100,verbose_eval=1000)

    # cv
    vl_pred = model.predict(vl_x, num_iteration=model.best_iteration)
    # 正解データ作成
    valid = transactions[(transactions['t_dat'] >= valid_start) & (transactions['t_dat'] <= valid_end)].copy()
    valid = valid[['customer_id', 'article_id']].drop_duplicates()
    valid = valid.groupby('customer_id')['article_id'].apply(list).reset_index()
    valid = valid.sort_values('customer_id').reset_index(drop=True)
    # 2値分類の出力を元に12個選定
    valid_pred = ml_valid[['customer_id', 'article_id']].copy()
    valid_pred['prob'] = vl_pred
    valid_pred = valid_pred.sort_values(['customer_id', 'prob'], ascending=False)
    valid_pred = valid_pred.groupby('customer_id').head(12)
    valid_pred = valid_pred.groupby('customer_id')['article_id'].apply(list).reset_index()
    valid_pred = valid_pred.sort_values('customer_id').reset_index(drop=True)
    assert(valid['customer_id'].tolist() == valid_pred['customer_id'].tolist())
    # MAP@12
    score = np.mean(apk(valid['article_id'].tolist(), valid_pred['article_id'].tolist()))

    return score

In [18]:
def objective(trial):
    cf = trial.suggest_int('cf', 0, 5)
    ctf = trial.suggest_int('ctf', 0, 5)
    atfd = trial.suggest_int('atfd', 0, 5)
    atfp = trial.suggest_int('atfp', 0, 5)
    pa = trial.suggest_int('pa', 0, 5)

    a = trial.suggest_int('a', 0, 5)
    w = trial.suggest_int('w', 0, 5)
    m = trial.suggest_int('m', 0, 5)
    y = trial.suggest_int('y', 0, 5)
    
    Ns['cf_a'] = cf * a
    Ns['ctf_a'] = ctf * a
    Ns['atfd_a'] = atfd * a
    Ns['atfp_a'] = atfp * a
    Ns['pa_a'] = pa * a

    Ns['cf_w'] = cf * w
    Ns['ctf_w'] = ctf * w
    Ns['atfd_w'] = atfd * w
    Ns['atfp_w'] = atfp * w
    Ns['pa_w'] = pa * w

    Ns['cf_m'] = cf * m
    Ns['ctf_m'] = ctf * m
    Ns['atfd_m'] = atfd * m
    Ns['atfp_m'] = atfp * m
    Ns['pa_m'] = pa * m

    Ns['cf_y'] = cf * y
    Ns['ctf_y'] = ctf * y
    Ns['atfd_y'] = atfd * y
    Ns['atfp_y'] = atfp * y
    Ns['pa_y'] = pa * y

    total_n = Ns['ctf_a'] + Ns['pa_a'] + \
              Ns['ctf_w'] + Ns['pa_w'] + \
              Ns['ctf_m'] + Ns['pa_m'] + \
              Ns['ctf_y'] + Ns['pa_y']

    if total_n > 12:
        score = run_train(transactions, articles, customers, Ns)
    else:
        score = 0.0
    message = f'{Ns}\n{score}'
    line_notify.send(message)

    return score

In [19]:
study = optuna.create_study(direction='maximize')
study.optimize(objective)

[I 2022-03-06 12:25:00,955] A new study created in memory with name: no-name-dc5ac115-bb8d-4b5e-967f-260751fd6fb0
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 12734, number of negative: 5101614
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.639817 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7545
[LightGBM] [Info] Number of data points in the train set: 5114348, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.002490 -> initscore=-5.993037
[LightGBM] [Info] Start training from score -5.993037
Training until validation scores don't improve for 100 rounds
[1000]	training's binary_logloss: 0.0138502	valid_1's binary_logloss: 0.014922
[2000]	training's binary_logloss: 0.0127597	valid_1's binary_logloss: 0.0149065
Early stopping, best iteration is:
[2424]	training's binary_logloss: 0.0123653	valid_1's binary_logloss: 0.0149039


[I 2022-03-06 12:43:41,796] Trial 0 finished with value: 0.017594331251912475 and parameters: {'cf': 0, 'ctf': 0, 'atfd': 5, 'atfp': 3, 'pa': 4, 'a': 3, 'w': 0, 'm': 4, 'y': 4}. Best is trial 0 with value: 0.017594331251912475.
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 8819, number of negative: 2463641
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.138911 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7695
[LightGBM] [Info] Number of data points in the train set: 2472460, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003567 -> initscore=-5.632487
[LightGBM] [Info] Start training from score -5.632487
Training until validation scores don't improve for 100 rounds
[1000]	training's binary_logloss: 0.0163767	valid_1's binary_logloss: 0.0186064
Early stopping, best iteration is:
[916]	training's binary_logloss: 0.0165357	valid_1's binary_logloss: 0.0186017


[I 2022-03-06 12:55:11,510] Trial 1 finished with value: 0.026153086501631787 and parameters: {'cf': 2, 'ctf': 1, 'atfd': 0, 'atfp': 3, 'pa': 2, 'a': 5, 'w': 0, 'm': 2, 'y': 1}. Best is trial 1 with value: 0.026153086501631787.
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 14374, number of negative: 4948045
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.543172 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7693
[LightGBM] [Info] Number of data points in the train set: 4962419, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.002897 -> initscore=-5.841327
[LightGBM] [Info] Start training from score -5.841327
Training until validation scores don't improve for 100 rounds
[1000]	training's binary_logloss: 0.0145848	valid_1's binary_logloss: 0.0159216
[2000]	training's binary_logloss: 0.0135165	valid_1's binary_logloss: 0.0158983
Early stopping, best iteration is:
[2123]	training's binary_logloss: 0.0133956	valid_1's binary_logloss: 0.0158973


[W 2022-03-06 13:15:30,609] Trial 2 failed because of the following error: AssertionError()
Traceback (most recent call last):
  File "/home/kaggler/.local/lib/python3.8/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_20560/1751778146.py", line 43, in objective
    score = run_train(transactions, articles, customers, Ns)
  File "/tmp/ipykernel_20560/1822577249.py", line 58, in run_train
    assert(valid['customer_id'].tolist() == valid_pred['customer_id'].tolist())
AssertionError


AssertionError: 

In [ ]:
print(study.best_params)

{'cf_a': 12, 'ctf_a': 9, 'atfd_a': 8, 'atfp_a': 12, 'pa_a': 2, 'cf_w': 8, 'ctf_w': 14, 'atfd_w': 23, 'atfp_w': 12, 'pa_w': 12, 'cf_m': 15, 'ctf_m': 10, 'atfd_m': 15, 'atfp_m': 11, 'pa_m': 5, 'cf_y': 19, 'ctf_y': 22, 'atfd_y': 3, 'atfp_y': 11, 'pa_y': 13}
